# Importing the libraries


In [62]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import math
from tabulate import tabulate

# Q5

In [63]:
class GridWorld:

  # Initilise the class
  def __init__(self,n,discount):
    self.grid = np.zeros((n,n))
    self.n = n
    self.discount = discount

  def update_value(self):
    for i in range(self.n):
      for j in range(self.n):
        new_value = 0

        # For state A
        if i==0 and j==1:
          new_value =  1*(10 + self.discount*self.grid[4][1])*1

        # For state B  
        elif i==0 and j==3:
          new_value =  1*(5 + self.discount*self.grid[2][3])*1

        # Other states  
        else :
          action = np.zeros(4)   # O - UP , 1 - DOWN , 2 - LEFT , 3 - RIGHT
          if i>0:
            action[0] = self.grid[i-1][j]
          if i<self.n-1:
            action[1] = self.grid[i+1][j]
          if j>0:
            action[2] = self.grid[i][j-1]
          if j<self.n-1:
            action[3] = self.grid[i][j+1]

          # Calculate the new value
          for k in range(4):
            if action[k]==0:
              new_value = new_value + (1/4)*(-1 + self.discount*self.grid[i][j])*1 
            else :
              new_value = new_value + (1/4)*(0 + self.discount*action[k])*1   
        self.grid[i][j] = new_value   

  def PrintGrid(self,decimal):
    grid_df = pd.DataFrame(np.round(self.grid,decimal))    
    print('Grid : ')
    print(grid_df)

In [64]:
iter = 20
G = GridWorld(n = 5,discount = 0.9)
for i in range(iter):
  G.update_value()
G.PrintGrid(decimal = 1)


Grid : 
     0    1    2    3    4
0  3.3  8.8  4.4  5.3  1.5
1  1.5  3.0  2.3  1.9  0.5
2  0.1  0.7  0.7  0.4 -0.4
3 -1.0 -0.4 -0.4 -0.6 -1.2
4 -1.9 -1.3 -1.2 -1.4 -2.0


# Q7

In [65]:
class GridWorld1:

  # Initilise the class
  def __init__(self,n,discount,delta):
    self.grid = np.zeros((n,n))
    self.prob = np.full((n,n,4),(1/4))
    self.n = n
    self.discount = discount
    self.delta = delta

  def Policy_Evaluation(self):
    while True:
      diff = 0
      for i in range(self.n):
        for j in range(self.n):
          new_value = 0
          old_value = self.grid[i][j]

          # For state A[0,1]
          if i==0 and j==1:  
            new_value =  1*(10 + self.discount*self.grid[4][1])*1

          # For state B[0,3]  
          elif i==0 and j==3:
            new_value =  1*(5 + self.discount*self.grid[2][3])*1

          # Other states  
          else :
            action = np.zeros(4)   # O - UP , 1 - DOWN , 2 - LEFT , 3 - RIGHT
            if i>0:
              action[0] = self.grid[i-1][j]
            if i<self.n-1:
              action[1] = self.grid[i+1][j]
            if j>0:
              action[2] = self.grid[i][j-1]
            if j<self.n-1:
              action[3] = self.grid[i][j+1]

            # Calculate the new value
            for k in range(4):
              if action[k]==0:
                new_value = new_value + self.prob[i][j][k]*(-1 + self.discount*self.grid[i][j])*1 
              else :
                new_value = new_value + self.prob[i][j][k]*(0 + self.discount*action[k])*1   
          self.grid[i][j] = new_value
          diff = max(diff,abs(old_value-new_value))    
      if diff<self.delta:
        break       

  def Policy_Improvement(self):
    policy_stable = True
    old_prob = np.copy(self.prob)

    for i in range(self.n):
      for j in range(self.n):
        if (i==0 and j==1) or (i==0 and j==3):
          continue
        else :
          action = np.zeros(4)   # O - UP , 1 - DOWN , 2 - LEFT , 3 - RIGHT
          if i>0:
            action[0] = self.grid[i-1][j]
          if i<self.n-1:
            action[1] = self.grid[i+1][j]
          if j>0:
            action[2] = self.grid[i][j-1]
          if j<self.n-1:
            action[3] = self.grid[i][j+1]

          # Calculate the new value
          new_prob = []
          for k in range(4):
            if action[k]==0:
              new_value = (-1 + self.discount*self.grid[i][j])*1 
            else :
              new_value = (0 + self.discount*action[k])*1  
            new_prob.append(new_value)
        
        new_prob = np.array(new_prob)    
        max_ind = np.where(new_prob == new_prob.max())[0]
        
        prob_value = 1/len(max_ind)
        for k in range(len(max_ind)):
          self.prob[i][j][max_ind[k]] = prob_value
        for k in [0,1,2,3]:
          if k not in max_ind:
            self.prob[i][j][k] = 0

    comp =  old_prob == self.prob     
    if(comp.all()==True):
      policy_stable = True
    else :
      policy_stable = False  

    return policy_stable  

  def Print_Grid(self,decimal):
    grid_df = pd.DataFrame(np.round(self.grid,decimal))    
    print('Grid : ')
    print(grid_df)

  def Print_Policy(self):
    policy = [["" for _ in range(self.n)] for _ in range(self.n)]
    direction = {0:"↑",1:"↓",2:"←",3:"→"}
    for i in range(self.n):
      for j in range(self.n): 
        t = policy[i][j]
        for k in range(4):
          if self.prob[i][j][k]>0:
            t = t + direction[k]+" "
        policy[i][j] = t
    policy_df = pd.DataFrame(policy)    
    print('Policy : ')
    print(tabulate(policy_df, headers='keys', tablefmt='psql'))



In [66]:
G = GridWorld1(n = 5,discount = 0.9,delta = 0.1e-10)
while True:
  G.Policy_Evaluation()
  policy_stable = G.Policy_Improvement()
  if policy_stable == True:
    break

G.Print_Grid(decimal = 1)
print()
G.Print_Policy()  


Grid : 
      0     1     2     3     4
0  22.0  24.4  22.0  19.4  17.5
1  19.8  22.0  19.8  17.8  16.0
2  17.8  19.8  17.8  16.0  14.4
3  16.0  17.8  16.0  14.4  13.0
4  14.4  16.0  14.4  13.0  11.7

Policy : 
+----+-----+---------+-----+---------+-----+
|    | 0   | 1       | 2   | 3       | 4   |
|----+-----+---------+-----+---------+-----|
|  0 | →   | ↑ ↓ ← → | ←   | ↑ ↓ ← → | ←   |
|  1 | ↑ → | ↑       | ↑ ← | ←       | ←   |
|  2 | ↑ → | ↑       | ↑ ← | ↑ ←     | ↑ ← |
|  3 | ↑ → | ↑       | ↑ ← | ↑ ←     | ↑ ← |
|  4 | ↑ → | ↑       | ↑ ← | ↑ ←     | ↑ ← |
+----+-----+---------+-----+---------+-----+


# Q9

In [67]:
class GridWorld2:

  # Initilise the class
  def __init__(self,n,discount,delta):
    self.grid = np.zeros((n,n))
    self.prob = np.full((n,n,4),(1/4))
    self.n = n
    self.discount = discount
    self.delta = delta

  def Policy_Evaluation(self):
    while True:
      diff = 0
      for i in range(self.n):
        for j in range(self.n):
          new_value = 0
          old_value = self.grid[i][j]

          # For state [0,0] and [3,3]
          if (i==0 and j==0) or (i==3 and j==3): 
            continue 
   
          # Other states  
          else :
            action = np.zeros(4)   # O - UP , 1 - DOWN , 2 - LEFT , 3 - RIGHT
            reward = np.full(4,-1)
            if i>0:
              action[0] = self.grid[i-1][j]
              if((i-1,j)==(0,0) or (i-1,j)==(3,3)):
                reward[0] = 0
            if i<self.n-1:
              action[1] = self.grid[i+1][j]
              if((i+1,j)==(0,0) or (i+1,j)==(3,3)):
                reward[1] = 0
            if j>0:
              action[2] = self.grid[i][j-1]
              if((i,j-1)==(0,0) or (i,j-1)==(3,3)):
                reward[2] = 0
            if j<self.n-1:
              action[3] = self.grid[i][j+1]
              if((i,j+1)==(0,0) or (i,j+1)==(3,3)):
                reward[3] = 0

            # Calculate the new value
            for k in range(4):
              if action[k]==0:
                new_value = new_value + self.prob[i][j][k]*(reward[k] + self.discount*self.grid[i][j])*1 
              else :
                new_value = new_value + self.prob[i][j][k]*(reward[k] + self.discount*action[k])*1   
          self.grid[i][j] = new_value
          diff = max(diff,abs(old_value-new_value))    
      if diff<self.delta:
        break       

  def Policy_Improvement(self):
    policy_stable = True
    old_prob = np.copy(self.prob)

    for i in range(self.n):
      for j in range(self.n):

        # For state [0,0] and [3,3]
        if (i==0 and j==0) or (i==3 and j==3): 
            continue 
        
        else :
          action = np.zeros(4)   # O - UP , 1 - DOWN , 2 - LEFT , 3 - RIGHT
          reward = np.full(4,-2)
          if i>0:
            action[0] = self.grid[i-1][j]
            if((i-1,j)==(0,0) or (i-1,j)==(3,3)):
              reward[0] = 0
          if i<self.n-1:
            action[1] = self.grid[i+1][j]
            if((i+1,j)==(0,0) or (i+1,j)==(3,3)):
              reward[1] = 0
          if j>0:
            action[2] = self.grid[i][j-1]
            if((i,j-1)==(0,0) or (i,j-1)==(3,3)):
              reward[2] = 0
          if j<self.n-1:
            action[3] = self.grid[i][j+1]
            if((i,j+1)==(0,0) or (i,j+1)==(3,3)):
              reward[3] = 0

          # Calculate the new value
          new_prob = []
          for k in range(4):
            if action[k]==0:
              new_value = (reward[k] + self.discount*self.grid[i][j])*1 
            else :
              new_value = (reward[k] + self.discount*action[k])*1  
            new_prob.append(new_value)
        
          new_prob = np.array(new_prob)    
          max_ind = np.where(new_prob == new_prob.max())[0]
          
          prob_value = 1/len(max_ind)
          for k in range(len(max_ind)):
            self.prob[i][j][max_ind[k]] = prob_value
          for k in [0,1,2,3]:
            if k not in max_ind:
              self.prob[i][j][k] = 0

    comp =  old_prob == self.prob     
    if(comp.all()==True):
      policy_stable = True
    else :
      policy_stable = False  

    return policy_stable  

  def Print_Grid(self,decimal):
    grid_df = pd.DataFrame(np.round(self.grid,decimal))    
    print('Grid : ')
    print(grid_df)

  def Print_Policy(self):
    policy = [["" for _ in range(self.n)] for _ in range(self.n)]
    direction = {0:"↑",1:"↓",2:"←",3:"→"}
    for i in range(self.n):
      for j in range(self.n): 
        t = policy[i][j]
        for k in range(4):
          if self.prob[i][j][k]>0:
            t = t + direction[k]+" "
        policy[i][j] = t
    policy_df = pd.DataFrame(policy)    
    print('Policy : ')
    print(tabulate(policy_df, headers='keys', tablefmt='psql'))



In [68]:
k = 0
print('------------ For k = 0 ---------------')
print('--------------------------------------')
G = GridWorld2(n = 4,discount = 0.9,delta = 0.01)
G.Print_Grid(decimal = 1)
print()
G.Print_Policy() 
while True:
  k = k + 1
  print()
  print('------------ For k = '+str(k),' --------------')
  print('--------------------------------------')
  G.Policy_Evaluation()
  policy_stable = G.Policy_Improvement()
  G.Print_Grid(decimal = 1)
  print()
  G.Print_Policy()  
  if policy_stable == True:
    break

print('--------------------------------------')
print('--------------------------------------')
print('v*(s) : ')
G.Print_Grid(decimal = 1)
print()
print('policy*(s) : ')
G.Print_Policy()  


------------ For k = 0 ---------------
--------------------------------------
Grid : 
     0    1    2    3
0  0.0  0.0  0.0  0.0
1  0.0  0.0  0.0  0.0
2  0.0  0.0  0.0  0.0
3  0.0  0.0  0.0  0.0

Policy : 
+----+---------+---------+---------+---------+
|    | 0       | 1       | 2       | 3       |
|----+---------+---------+---------+---------|
|  0 | ↑ ↓ ← → | ↑ ↓ ← → | ↑ ↓ ← → | ↑ ↓ ← → |
|  1 | ↑ ↓ ← → | ↑ ↓ ← → | ↑ ↓ ← → | ↑ ↓ ← → |
|  2 | ↑ ↓ ← → | ↑ ↓ ← → | ↑ ↓ ← → | ↑ ↓ ← → |
|  3 | ↑ ↓ ← → | ↑ ↓ ← → | ↑ ↓ ← → | ↑ ↓ ← → |
+----+---------+---------+---------+---------+

------------ For k = 1  --------------
--------------------------------------
Grid : 
     0    1    2    3
0  0.0 -9.0 -9.3 -9.5
1 -9.0 -9.2 -9.4 -9.4
2 -9.3 -9.4 -9.2 -9.0
3 -9.5 -9.4 -9.0  0.0

Policy : 
+----+---------+-----+-----+---------+
|    | 0       | 1   | 2   | 3       |
|----+---------+-----+-----+---------|
|  0 | ↑ ↓ ← → | ←   | ←   | ←       |
|  1 | ↑       | ↑   | ←   | ↓       |
|  2 | ↑      